# Credits

This is heavily influenced or copied from https://github.com/pytorch/tutorials

# Autograd: automatic differentiation

Central to all neural networks in PyTorch is the ``autograd`` package.
Let’s first briefly visit this, and we will then go to training our first neural network.

The `autograd` package provides automatic differentiation for all operations on Tensors.
It is a define-by-run framework, which means that your backprop is defined by how your code is run, and that every single iteration can be different.

Let us see this in more simple terms with some examples.

## 1. Tensor

`torch.Tensor` is the central class of the package. Setting the attribute `.requires_grad` to `True` will make the tensor "record" all operations on it. When you finish your computation you can call `.backward()` and have all the gradients computed automatically. The gradient for this tensor will be accumulated into the `.grad` attribute.

![autograd.Variable](https://github.com/markusOlsen1/Deep_Learning/blob/master/static_files/autograd-variable.png?raw=1)

There’s one more class which is very important for autograd implementation - a `Function`.

`Tensor` and `Function` are interconnected and build up an acyclic graph, that encodes a complete history of computation. Each tensor has a `.grad_fn` attribute that references a `Function` that has created the `Tensor` (except for Tensors created by the user - their `grad_fn` is `None`).

If you want to compute the derivatives, you can call `.backward()` on a Tensor. If `Tensor` is a scalar (i.e. it holds a one element data), you don’t need to specify any arguments to backward(), however if it has more elements, you need to specify a `gradient` argument that is a tensor of matching shape.

Create a tensor

In [1]:
import torch

In [2]:
x = torch.ones(2, 2, requires_grad=True)
print(x)

tensor([[1., 1.],
        [1., 1.]], requires_grad=True)


Do a tensor operation:

In [16]:
y = x + 2
print(y)

tensor([[2.6082, 2.4849, 2.3613, 2.8094, 2.7117],
        [2.3832, 2.9790, 2.9960, 2.1530, 2.7491],
        [2.4528, 2.3053, 2.3982, 2.7875, 2.1708],
        [2.7501, 2.6755, 2.1672, 2.8830, 2.0474],
        [2.5067, 2.0273, 2.6823, 2.4852, 2.8941]], grad_fn=<AddBackward0>)


`y` was created as a result of an operation, so it has a `grad_fn`.

In [17]:
print(y.grad_fn)

Do more operations on y

In [18]:
z = y * y * 3
out = z.mean()

print(z)
print(out)

tensor([[20.4074, 18.5237, 16.7273, 23.6790, 22.0607],
        [17.0395, 26.6237, 26.9278, 13.9066, 22.6720],
        [18.0487, 15.9429, 17.2544, 23.3105, 14.1377],
        [22.6899, 21.4745, 14.0908, 24.9342, 12.5752],
        [18.8501, 12.3297, 21.5834, 18.5293, 25.1269]], grad_fn=<MulBackward0>)
tensor(19.5778, grad_fn=<MeanBackward0>)


# Assignments

1. Create a Tensor that `requires_grad` of size (5, 5)
2. Sum the values in the Tensor

In [19]:
x_ten = torch.rand(5,5, requires_grad = True)
x_ten.sum()

tensor(10.4659, grad_fn=<SumBackward0>)

## 2. Gradients

Let’s backprop now. Because `out` contains a single scalar, `out.backward()` is equivalent to `out.backward(torch.tensor([1.0]))`

In [20]:
out.backward()


Print gradients d(out)/dx

In [21]:
print(x.grad)

tensor([[0.6260, 0.5964, 0.5667, 0.6743, 0.6508],
        [0.5720, 0.7150, 0.7190, 0.5167, 0.6598],
        [0.5887, 0.5533, 0.5756, 0.6690, 0.5210],
        [0.6600, 0.6421, 0.5201, 0.6919, 0.4914],
        [0.6016, 0.4865, 0.6437, 0.5965, 0.6946]])


You should have a matrix of `4.5`. Let’s denote the tensor `out` with $o$.

We have:
$o = \frac{1}{4}\sum_i z_i$, $z_i = 3(x_i+2)^2$ and $z_i\bigr\rvert_{x_i=1} = 27$.

Therefore, $\frac{\partial o}{\partial x_i} = \frac{3}{2}(x_i+2)$,
hence $\frac{\partial o}{\partial x_i}\bigr\rvert_{x_i=1} = \frac{9}{2} = 4.5$.

You can do many crazy things with autograd!

In [8]:
x = torch.randn(3, requires_grad=True)

y = x * 2
while y.data.norm() < 1000:
    y = y * 2

print(y)

tensor([-1206.2593,  -266.5599,  -986.0686], grad_fn=<MulBackward0>)


In [9]:
gradients = torch.FloatTensor([0.1, 1.0, 0.0001])
y.backward(gradients)

print(x.grad)

tensor([1.0240e+02, 1.0240e+03, 1.0240e-01])


**Read later** \
*Documentation* \
`Tensor`: https://pytorch.org/docs/stable/tensors.html \
`Function`: http://pytorch.org/docs/autograd

# Assignments

1. Define a tensor and set `requires_grad` to `True`
3. Multiply the tensor by 2 and assign the result to a new python variable (i.e. `x = result`)
4. Sum the variable's elements and assign to a new python variable
5. Print the gradients of all the variables
6. Now perform a backward pass on the last variable (NOTE: for each new python variable that you define, call `.retain_grad()`)
7. Print all gradients again
  - what did you notice?